In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet

In [2]:
path_data = '../data/ventas.csv'

In [4]:
df = pd.read_csv(path_data, encoding='iso-8859-1')
df.columns = ['fecha', 'codigo', 'descripcion', 'precio_unitario', 
              'costo_unitario', 'cantidad_vendida']

In [5]:
dict_sku = df[['codigo','descripcion']].drop_duplicates().reset_index(drop=True).set_index('codigo').to_dict()['descripcion']

In [6]:
df_general = df.copy()
df_general['precio_total'] = df_general['precio_unitario'] * df_general['cantidad_vendida']
df_general['costo_total']  = df_general['costo_unitario'] * df_general['cantidad_vendida']
df_general = df_general.groupby(['codigo','fecha']).agg({'cantidad_vendida':'sum',
                                                     'precio_total':'sum',
                                                     'costo_total':'sum'}).reset_index()

In [7]:
# Para información de la semana 2024-08 de todo los sku id
df_total_202408 = df_general[df_general['fecha']=='2024-08'].copy()

# Para información menos a la semana 2024-08
df_total = df_general[df_general['fecha']!='2024-08'].copy()

In [8]:
import datetime

# año-semana a datetime
def year_week_to_date(year_week):
    year, week = map(int, year_week.split('-'))
    first_day_of_year = datetime.date(year, 1, 1)
    base_week_day = first_day_of_year - datetime.timedelta(days=first_day_of_year.weekday())
    return base_week_day + datetime.timedelta(weeks=(week))

# datetime a año-semana
def date_to_year_week(date):
    if isinstance(date, str):
        date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
    year = date.year
    week = date.isocalendar().week
    return f"{year}-{week:02d}"


In [14]:
from sklearn.metrics import mean_absolute_error

#  MASE (Error Absoluto Medio de Escala de Error)
def mase(actual, forecast, seasonality):
    actual_diff = np.diff(actual, n=seasonality)
    forecast_diff = np.diff(forecast, n=seasonality)
    scaled_error = np.mean(np.abs(actual_diff - forecast_diff))
    naive_error = np.mean(np.abs(actual_diff[seasonality:] - actual_diff[:-seasonality]))
    mase_value = scaled_error / naive_error
    return mase_value

# Pronóstico en el Intervalo (PI)
def pi(actual, lower_bound, upper_bound):
    within_interval = np.sum((actual >= lower_bound) & (actual <= upper_bound))
    percentage_within_interval = within_interval / len(actual)
    return percentage_within_interval

# MAPE (Error Porcentual Absoluto Medio)
def mean_absolute_porcentual_error(real, pred):
    real, pred = np.array(real), np.array(pred)
    return np.mean(np.abs((real - pred) / real)) * 100

# Metricas
def metrics(forecast, var_target):
    df = forecast.copy()
    df = df.dropna(subset=[var_target], axis=0)

    ls_mase = list()
    for s in range(1, 13):
        ls_mase.append( mase(df[var_target], df['yhat'], s) )

    pi_score = pi(df[var_target], df['yhat_lower'], df['yhat_upper'])
    mae = mean_absolute_error(df[var_target], df['yhat'])
    mape = mean_absolute_porcentual_error(df[var_target], df['yhat'])

    print(f'METRICS -> MAPE: {np.round(mape,4)}, MAE: {np.round(mae,4)}, MASE: {np.round(np.mean(ls_mase),4)}, PI: {np.round(pi_score,4)}')

    return [np.round(mape,4), np.round(mae,4), np.round(np.mean(ls_mase),4), np.round(pi_score,4)]

In [15]:
# Creacion de modelos por SKU
models = {}
metrics_save = {}
sku_next_week_df = {}
sku_next_week_full = {}
for sku_id in dict_sku.keys():
    print('=====================================================================================')
    print('SKU ID:', sku_id)
    data_sku = df_total[df_total['codigo']==sku_id].copy()
    
    data_sku['date_formatted'] = data_sku['fecha'].apply(year_week_to_date)
    
    # train = data_sku[:int(0.80*(len(data_sku)))]
    # valid = data_sku[int(0.80*(len(data_sku))):]
    
    # valid['date_formatted'] = valid['fecha'].apply(year_week_to_date)
    
    # train_prophet = pd.DataFrame()
    # train_prophet['ds'] = train['date_formatted']
    # train_prophet['y'] = train['cantidad_vendida']
    
    # Entremiento del modelo
    train_prophet = pd.DataFrame()
    train_prophet['ds'] = data_sku['date_formatted']
    train_prophet['y'] = data_sku['cantidad_vendida']    
    
    # Modelo Prophet con estacionalidades anual, semanal y diaria (ya que los datos son semanales)
    model = Prophet(yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True, seasonality_mode = 'multiplicative')
    model.fit(train_prophet)
    
    # Predicción de la siguiente semana
    future = model.make_future_dataframe(periods = 1, freq = 'w') 
    forecast = model.predict(future)

    forecast_final = forecast.set_index('ds')
    forecast_final['cantidad_vendida'] = data_sku.set_index(['date_formatted'])['cantidad_vendida']

    # Métricas, se usaron las métricas de MAPE, MAE, MASE y PI
    mt = metrics(forecast_final, 'cantidad_vendida')
    
    # Plot de los resultados del modelo
    # fig = model.plot(forecast,figsize=(20, 5), include_legend=True)
    # plt.plot(valid.set_index(['date_formatted'])['cantidad_vendida'], label='Valid', color = 'red', linewidth = 2)
    # plt.title(f'SKU ID: {sku}')
    # plt.show()
    
    # Guardar resultados
    pred_next_week = forecast_final[['yhat','yhat_lower','yhat_upper']].round(0).iloc[[-1]].T.to_dict()
    next_timestamp = list(pred_next_week.keys())[0]
    sku_next_week_df[sku_id] = {
                                'fecha': date_to_year_week(next_timestamp),
                                'pred': pred_next_week[next_timestamp]['yhat'],
                                'pred_min': pred_next_week[next_timestamp]['yhat_lower'],
                                'pred_max': pred_next_week[next_timestamp]['yhat_upper']
                            }
    sku_next_week_full[sku_id] = {
                                'fecha': date_to_year_week(next_timestamp),
                                'pred': pred_next_week[next_timestamp]['yhat'],
                                'pred_min': pred_next_week[next_timestamp]['yhat_lower'],
                                'pred_max': pred_next_week[next_timestamp]['yhat_upper'],
                                'meta-data': {
                                    'train-metrics': {
                                        'mape': mt[0],
                                        'mae': mt[1],
                                        'mase': mt[2],
                                        'pi': mt[3]
                                    }
                                }
                            }
    models[sku_id] = model
    metrics_save[sku_id] = {
                        'mape': mt[0],
                        'mae': mt[1],
                        'mase': mt[2],
                        'pi': mt[3]
                    }


02:20:22 - cmdstanpy - INFO - Chain [1] start processing
02:20:22 - cmdstanpy - INFO - Chain [1] done processing


SKU ID: 10096015


02:20:22 - cmdstanpy - INFO - Chain [1] start processing
02:20:22 - cmdstanpy - INFO - Chain [1] done processing


METRICS -> MAPE: 26.2037, MAE: 4.4792, MASE: 0.7073, PI: 0.8288
SKU ID: 10108320


02:20:22 - cmdstanpy - INFO - Chain [1] start processing
02:20:22 - cmdstanpy - INFO - Chain [1] done processing


...

METRICS -> MAPE: 47.1371, MAE: 109.2864, MASE: 0.6418, PI: 0.8108
SKU ID: 108930


02:20:24 - cmdstanpy - INFO - Chain [1] start processing
02:20:24 - cmdstanpy - INFO - Chain [1] done processing


METRICS -> MAPE: 43.7205, MAE: 2.3638, MASE: 0.7018, PI: 0.8018
SKU ID: 109885
METRICS -> MAPE: 20.6449, MAE: 27.1156, MASE: 0.726, PI: 0.75
SKU ID: 10121151


02:20:24 - cmdstanpy - INFO - Chain [1] start processing
02:20:24 - cmdstanpy - INFO - Chain [1] done processing


METRICS -> MAPE: 123.4121, MAE: 38.6272, MASE: 0.7041, PI: 0.8


In [11]:
forecast_final

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,...,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,cantidad_vendida
ds,,,,,,,,,,,,,,,,,,,,,
2022-01-03,13.058526,19.346266,33.247695,13.058526,13.058526,1.027352,1.027352,1.027352,1.017461,1.017461,...,0.770514,0.770514,-0.780404,-0.780404,-0.780404,0.0,0.0,0.0,26.345069,24.0
2022-01-10,12.880406,20.945612,35.475431,12.880406,12.880406,1.027352,1.027352,1.027352,1.208820,1.208820,...,0.770514,0.770514,-0.589045,-0.589045,-0.589045,0.0,0.0,0.0,28.450502,29.0
2022-01-17,12.702285,24.885178,39.469470,12.702285,12.702285,1.027352,1.027352,1.027352,1.498700,1.498700,...,0.770514,0.770514,-0.299165,-0.299165,-0.299165,0.0,0.0,0.0,31.739203,27.0
2022-01-24,12.524165,25.212335,39.036469,12.524165,12.524165,1.027352,1.027352,1.027352,1.572885,1.572885,...,0.770514,0.770514,-0.224980,-0.224980,-0.224980,0.0,0.0,0.0,32.223239,38.0
2022-01-31,12.346045,23.207629,37.490648,12.346045,12.346045,1.027352,1.027352,1.027352,1.436019,1.436019,...,0.770514,0.770514,-0.361847,-0.361847,-0.361847,0.0,0.0,0.0,30.075196,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-29,8.128302,13.404205,28.171103,8.128302,8.128302,1.027352,1.027352,1.027352,1.495265,1.495265,...,0.770514,0.770514,-0.302601,-0.302601,-0.302601,0.0,0.0,0.0,20.282264,16.0
2024-02-05,8.183625,11.633725,26.554805,8.183625,8.183625,1.027352,1.027352,1.027352,1.348398,1.348398,...,0.770514,0.770514,-0.449467,-0.449467,-0.449467,0.0,0.0,0.0,19.218411,18.0
2024-02-12,8.238948,11.861467,26.395944,8.238948,8.238948,1.027352,1.027352,1.027352,1.335780,1.335780,...,0.770514,0.770514,-0.462085,-0.462085,-0.462085,0.0,0.0,0.0,19.244371,12.0
